# Modeling

1. This notebook implements 3 models
    *   A lineal regression model as baseline
    *   A tree-based as potential candidate and also as a pipeline to do Feature importance
    *   A MLP as to use as the main regression model|

### Training and test datasets preparation

In [106]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.centroids = None


    def __compute_distance(self, point, centroid):
        return np.sqrt((point[0] - centroid[0])**2 + (point[1] - centroid[1])**2)

    def __distance_to_closest_centroid(self, row):
        distances = [self.__compute_distance([row['latitude'], row['longitude']], centroid) for centroid in self.centroids]
        return min(distances)

    def __get_distance_to_high_value_area(self, X):
        if self.centroids is None:
            raise ValueError('Fit the model first before transforming the data')
        
        X['distance_to_high_value_area'] = X.apply(lambda row: self.__distance_to_closest_centroid(row), axis=1)
        return X

    # Fit the KMeans clustering model for the distance to high value area feature
    def fit(self, X, y=None, num_clusters=2):
        threshold = X['median_house_value'].quantile(0.80)
        high_value_homes = X[X['median_house_value'] > threshold]

        kmeans = KMeans(n_clusters=num_clusters, random_state=10)
        high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])
        self.centroids = kmeans.cluster_centers_

        return self
    
    def transform(self, X):
        X['bedrooms_per_room'] = X['total_bedrooms'] / X['total_rooms']
        X['population_per_household'] = X['population'] / X['households']
        X['rooms_per_household'] = X['total_rooms'] / X['households']
        X['income_per_household'] = X['median_income'] / X['households']
        X['income_per_bedroom'] = X['median_income'] / X['total_bedrooms']
        X['income_per_room'] = X['median_income'] / X['total_rooms']
        X['age_of_population'] = X['housing_median_age'] / X['population']
        X['age_of_households'] = X['housing_median_age'] / X['households']
        X['population_density'] = X['population'] / X['total_rooms']
        X['bedrooms_per_population'] = X['total_bedrooms'] / X['population']
        X['is_capped'] = X['median_house_value'] == 500000
        

        self.__get_distance_to_high_value_area(X)
        
        return X

# Custom transformer to drop a column    
class DropColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.column_name in X.columns:
            X = X.drop(columns=[self.column_name])
        return X
    
housing_data = pd.read_csv("data/housing.csv")
train_set, test_set = train_test_split(housing_data, test_size=0.3, random_state=10)

# Keep the original labels for training and testing
y_train = train_set['median_house_value'].to_numpy()
y_test = test_set['median_house_value'].to_numpy()

num_attribs = train_set.columns.tolist()
num_attribs.remove('ocean_proximity')
cat_attribs = ['ocean_proximity']

feature_eng = FeatureEngineering()
feature_eng.fit(train_set)

num_pipeline = Pipeline([
    ('feature_eng', feature_eng),
    ('drop_column', DropColumnTransformer('median_house_value')),
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])

X_train = pipeline.fit_transform(train_set)
X_test = pipeline.transform(test_set)

print(X_train.shape)
print(X_test.shape)

/tmp/ipykernel_28590/3990379248.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])
/tmp/ipykernel_28590/3990379248.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])


(14448, 25)
(6192, 25)


### Train and test a linear regression model

To evaluate:
1. The Root Mean Squared Error is used for the training error 
2. N-fold cross validation with RMSE is used, with 10 folds for the validation error

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_predictions = lin_reg.predict(X_train)

lin_rmse = np.sqrt(mean_squared_error(y_train, lin_predictions))
lin_scores = cross_val_score(lin_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

print('Linear Regression RMSE:', lin_rmse)
print('Linear Regression RMSE Scores:', lin_rmse_scores)
print('Linear Regression RMSE Mean:', lin_rmse_scores.mean())
print('Linear Regression RMSE Standard Deviation:', lin_rmse_scores.std())    


Linear Regression RMSE: 65512.19687815971
Linear Regression RMSE Scores: [65491.56395045 62939.19496873 64423.00782818 65956.13642558
 67187.4882399  66054.65590843 64872.79467416 79491.74134393
 65726.6694338  70087.43228025]
Linear Regression RMSE Mean: 67223.06850533944
Linear Regression RMSE Standard Deviation: 4457.011346877264


To note:

1. It seems that the model is underfitting as the error is around $67k
2. We could try to:
    * Use a more powerful model
    * Add more features that give more signal

### Training a decision tree model

In [43]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
tree_pred = tree_reg.predict(X_train)

tree_rmse = np.sqrt(mean_squared_error(y_train, tree_pred))
scores = cross_val_score(tree_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

print(f"RMSE: {tree_rmse}")
print(f"Scores: {tree_rmse_scores}")
print(f"Mean: {tree_rmse_scores.mean()}")
print(f"Standard Deviation: {tree_rmse_scores.std()}")


RMSE: 0.0
Scores: [67342.37070237 64333.83323831 69251.34131217 65383.86508618
 70360.5906065  66883.83561217 65856.91159433 66622.5999637
 67069.65300694 65451.15859048]
Mean: 66855.61597131658
Standard Deviation: 1727.9332081551922


To note:

1. Decision tree regressor is overfitting the data (Training RMSE = 0)
2. Performance is a little bit worse than the lineal regressor model when cross-validated

### Training a random forest model

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
forest_pred = forest_reg.predict(X_train)

forest_rmse = np.sqrt(mean_squared_error(y_train, forest_pred))
scores = cross_val_score(forest_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)

print(f"RMSE: {forest_rmse}")
print(f"Scores: {forest_rmse_scores}")
print(f"Mean: {forest_rmse_scores.mean()}")
print(f"Standard Deviation: {forest_rmse_scores.std()}")


RMSE: 17607.54218024274
Scores: [46340.26170842 44399.81293542 47564.20915831 46293.68816815
 48362.21875891 48347.57464029 46158.84466152 48441.52319964
 46878.14361248 49678.66839416]
Mean: 47246.494523731606
Standard Deviation: 1450.9680355512453


To note:

1. The model stills overfit a lot but given that random forest is an ensemble mechanism, this helps to generalize and regularize, the overfitting is not as bad as the decision tree
2. We could limit the 'max_depth', 'min_samples_split' and 'min_samples_leaf' to force the model to limit the complexity of the tree

### Multi Layered Perceptron (MLP)

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'{device=}')

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

# Custom wrapper class to use PyTorch model with scikit-learn cross-validation
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model, epochs=100, lr=0.01):
        self.model = model.to(device)
        self.epochs = epochs
        self.lr = lr

    def fit(self, X, y):
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        self.model.train()
        for _ in range(self.epochs):
            optimizer.zero_grad()
            outputs = self.model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        return self

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32).to(device)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X).cpu().numpy()
        return predictions.flatten()

# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 500
output_size = 1

mlp = MLP(input_size, hidden_size, output_size)
mlp_regressor = PyTorchRegressor(model=mlp, epochs=5000, lr=0.1)
mlp_regressor.fit(X_train, y_train)
mlp_predictions = mlp_regressor.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, mlp_predictions))
scores = cross_val_score(mlp_regressor, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
rmse_scores = np.sqrt(-scores)

print(f"RMSE: {rmse}")
print(f"Scores: {rmse_scores}")
print(f"Mean: {rmse_scores.mean()}")
print(f"Standard Deviation: {rmse_scores.std()}")

device=device(type='cuda')
RMSE: 33557.09451622558
Scores: [45475.11838989 50040.79888641 45579.42498517 44773.79124532
 47554.97488784 50931.20698013 45988.3425077  48157.77281182
 45890.09238523 47036.40736751]
Mean: 47142.79304470196
Standard Deviation: 1941.6837196220183


To note:

1. The MLP has slightly better performance to the Random Forest regressor, and the overfitting is not as bad as random Forest.
2. On GPU, the training time is the same as Random Forest, so this is a constrain.
3. Hyperparameter optimization of this model was done manually (epochs and hidden layer sizes)

## Hyperparamter optimization and Feature importance

1. Do hyperparameter optimization for the Random Forest regressor
2. Do Feature Importance to get an idea of the performance of the features

In [66]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create the grid search object
grid_search = GridSearchCV(estimator=forest_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)

print('Best Parameters:', best_params)
print('Best RMSE Score:', best_score)


/home/iox/anaconda3/envs/ai/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "/home/iox/anaconda3/envs/ai/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/iox/anaconda3/envs/ai/lib/python3.9/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/iox/anaconda3/envs/ai/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constrai

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 30}
Best RMSE Score: 50100.13279895036


To note:

1. After doing grid search on the forest regressor, the best loss is around 50k, slightly worse than the MLP with manual tuning

In [109]:
extra_attributes = ['bedrooms_per_room', 
                    'population_per_household', 
                    'rooms_per_household', 
                    'income_per_household', 
                    'income_per_bedroom', 
                    'income_per_room', 
                    'age_of_population', 
                    'age_of_households', 
                    'population_density', 
                    'bedrooms_per_population', 
                    'is_capped', 
                    'distance_to_high_value_area']

feature_importances = grid_search.best_estimator_.feature_importances_
cat_encoder = pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attributes + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)


[(0.22873440019471142, 'median_income'),
 (0.10909431480194261, '<1H OCEAN'),
 (0.10744088904140649, 'is_capped'),
 (0.08787105390446648, 'age_of_households'),
 (0.0568932893333759, 'median_house_value'),
 (0.056617735895357686, 'population_per_household'),
 (0.05233745282798575, 'latitude'),
 (0.05158124681506645, 'bedrooms_per_room'),
 (0.04599225591097741, 'longitude'),
 (0.036920091891611176, 'population_density'),
 (0.02429162542348582, 'housing_median_age'),
 (0.019281229771860104, 'income_per_household'),
 (0.018462488770107508, 'total_rooms'),
 (0.01593517586685576, 'rooms_per_household'),
 (0.01332673513865565, 'income_per_room'),
 (0.011899464100642602, 'age_of_population'),
 (0.011775810667396325, 'income_per_bedroom'),
 (0.011187974658039827, 'households'),
 (0.010714615483946578, 'total_bedrooms'),
 (0.010362703092917685, 'population'),
 (0.008188268070129269, 'distance_to_high_value_area'),
 (0.007105449369700127, 'NEAR BAY'),
 (0.002397682757945338, 'ISLAND'),
 (0.001570

To note:

1. distance_to_high_value_area is not a very important feature, and to get this feature a clustering model has to be trained, so this feature is dropped. The hypothesis is that '<1H OCEAN>' is highly correlated with distance_to_high_value_area and there is redundancy in these 2 features
2. A threshold of 0.01 is going to be use to filter out the least important features

## Conclusions

1. The MLP will be used for the e2e system. The reasons are:
    * Better performance compared to RF (though limited optimization in the RF regressor was done)
    * Scaling a NN is more straight forward (Data scaling, Model parallelism/sharding, distributed training/inference, quamtization, etc)
    * As more data is available, the NN architecture can be arbitrarily more complex to fit the data.
2. The feature engineering pipeline will be reworked due to the feature importance findings.